In [185]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [186]:
from classes.grid import Grid
from queue import PriorityQueue
import re

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [187]:
class MemorySpace(Grid):
    def __init__(self, grid, falling_bytes):
        super().__init__(grid)
        self.start_pos = (0,0)
        self.end_pos = (self.num_rows-1,self.num_cols-1)
        self.corrupted_tiles = set()
        self.falling_bytes = falling_bytes
    
    def next_byte_fall(self, times=1, return_byte_coords=False):
        '''
        Causes the next 'times' bytes to fall onto the grid
        '''
        for _ in range(times):
            next_falling_byte = self.falling_bytes.pop(0)
            self.grid[next_falling_byte[0]][next_falling_byte[1]] = '#'
        
        if return_byte_coords:
            return next_falling_byte
    
    def shortest_path(self, ):
        '''
        Finds the shortest path from start_pos -> end_pos (default top left to bottom right)
        '''

        seen = set()
        # A node in this grid is simply just a tile, (easier than Q16!),
        # and node neighbours are '.' tiles that are adjacent

        nodes_to_visit = PriorityQueue()
        nodes_to_visit.put((0, self.start_pos))

        while(nodes_to_visit.qsize() > 0):
            cur_node = nodes_to_visit.get()
            cur_node_cost = cur_node[0]
            cur_node_coords = cur_node[1]

            # Get neighbours

            neighbours = set()
            for dir in ['^','<','v','>']:
                adj_tile = self.get_relative(cur_node_coords[0], cur_node_coords[1], dir)
                if adj_tile[2] == '.' and adj_tile[0] != -1:
                    neighbours.add((adj_tile[0],adj_tile[1]))
            
            for neighbour in neighbours:
                if neighbour in seen:
                    continue

                neighbour_cost = cur_node_cost + 1
                if neighbour == self.end_pos:
                    return neighbour_cost
                
                nodes_to_visit.put((neighbour_cost,neighbour))
                seen.add(neighbour)
        return 'Failed'
        
                


In [188]:
falling_bytes_info = input.split('\n')

# Extract x and y coords of falling bytes in order

falling_bytes = []

for falling_byte in falling_bytes_info:
    falling_byte_x = int(re.findall(r'^\d+', falling_byte)[0])
    falling_byte_y = int(re.findall(r'\d+$', falling_byte)[0])
    falling_bytes.append((falling_byte_x,falling_byte_y))


# Initialise empty grid

if input == example:
    num_rows = 7
    num_cols = 7
    falls = 12
elif input == puzzle:
    num_rows = 71
    num_cols = 71
    falls = 1024

memory_space = MemorySpace(['.'*num_cols]*num_rows, falling_bytes)
#memory_space.print_grid()

# todo: make inherited class from grid that includes the falling bytes order and add a method that causes the next byte to fall,
# then add method that implements pathfinding alg for current state of grid

memory_space.next_byte_fall(times=falls)
#memory_space.print_grid()

memory_space.shortest_path()

454

# Part 2

In [189]:
import tqdm

falling_bytes_info = input.split('\n')

# Extract x and y coords of falling bytes in order

falling_bytes = []

for falling_byte in falling_bytes_info:
    falling_byte_x = int(re.findall(r'^\d+', falling_byte)[0])
    falling_byte_y = int(re.findall(r'\d+$', falling_byte)[0])
    falling_bytes.append((falling_byte_x,falling_byte_y))


# Initialise empty grid

if input == example:
    num_rows = 7
    num_cols = 7
elif input == puzzle:
    num_rows = 71
    num_cols = 71

memory_space = MemorySpace(['.'*num_cols]*num_rows, falling_bytes)
#memory_space.print_grid()

# todo: make inherited class from grid that includes the falling bytes order and add a method that causes the next byte to fall,
# then add method that implements pathfinding alg for current state of grid

for i in tqdm.tqdm(range(2800,len(falling_bytes))):
    this_falling_bytes = falling_bytes.copy()
    memory_space = MemorySpace(['.'*num_cols]*num_rows, this_falling_bytes)
    cur_falling_byte = memory_space.next_byte_fall(times=i,return_byte_coords=True)
    shortest_path = memory_space.shortest_path()
    #print(shortest_path)
    if memory_space.shortest_path() == 'Failed':
        print(i, cur_falling_byte)
        break
    
#memory_space.print_grid()



  8%|▊         | 50/650 [00:00<00:04, 141.75it/s]

2850 (8, 51)
